In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import mplfinance as mpf
import os
from pandas import to_datetime

In [2]:
data_2019 = pd.read_excel(r'C:\Users\USER\Paper\2019.xlsx')
data_2020 = pd.read_excel(r'C:\Users\USER\Paper\2020.xlsx')
data_2021 = pd.read_excel(r'C:\Users\USER\Paper\2021.xlsx')
data_2022 = pd.read_excel(r'C:\Users\USER\Paper\2022.xlsx')

In [3]:
data = pd.concat([data_2019, data_2020,data_2021,data_2022]).reset_index(drop=True)

In [4]:
df = data.rename(columns={'年月日':'Date',
                          '開盤價(元)':'Open',
                          '最高價(元)':'High',
                          '最低價(元)':'Low',
                          '收盤價(元)':'Close',})

In [5]:
df_drop = df.drop_duplicates('代號',keep='first').reset_index(drop=True)
df_drop['代號'].values

array(['Y9999', 'OTC99', '1101', ..., '6859', '6874', '8227'],
      dtype=object)

In [6]:
df['Date'] = pd.to_datetime(df['Date'])

In [7]:
code_draw = []
for code in df_drop['代號'].values:
    data = df[df['代號']==code].reset_index(drop=True) 
    for i in range(len(data)-120):
        stock = data[i:i+120]   
        if (stock['Low'].min() /  stock['High'].max() - 1) <= -0.6 and (code not in code_draw):
            code_draw.append(code)

In [8]:
len(code_draw)

1090

In [9]:
drawn_stocks = []
figure_paths = []
upper = 350
lower = 0
for code in code_draw:
    draw_df = df[df['代號'] == code].reset_index(drop=True) 

    for i in range(len(draw_df)-120):
        
        stock = draw_df[i:i+120]
        stock = stock.iloc[:,2:].reset_index(drop=True)
        for j in range(len(draw_df.index[draw_df['High'] == stock['High'].max()])):
            max_loc = draw_df.index[draw_df['High'] == stock['High'].max()][j]
            min_loc = max_loc - 120

            if (stock['Low'].min()/stock['High'].max()-1)<=-0.3 and (stock['High'].idxmax()<stock['Low'].idxmin()):
                draw = draw_df[min_loc:max_loc].set_index('Date')
                if draw['High'].max() != draw_df['High'].loc[max_loc]:
                    continue 
                if any(draw.equals(existing_draw) for existing_draw in drawn_stocks):
                    continue
                else:

                    if len(draw) == 120:
                        if not os.path.exists(code):
                            os.makedirs(code)
                        mc = mpf.make_marketcolors(up='r', down='g', inherit=True)
                        s  = mpf.make_mpf_style(base_mpf_style='yahoo', marketcolors=mc)
                        mpf.plot(draw,
                                type='candle',
                                volume=False,
                                figratio=(15,8),
                                figscale=1,
                                style=s,
                                axisoff = True,
                                savefig=os.path.join(code, code + '_' + str(i) + '_' + str(j) + '.png'))    
                        figure_paths.append(os.path.join(code, code + '_' + str(i) + '_' + str(j) + '.png'))
                drawn_stocks.append(draw)

In [10]:
figure_paths = pd.DataFrame({'File Paths': figure_paths})
figure_paths.to_csv('Loser_path.csv')
#figure_paths.to_csv('Loser_path' + '_' + str(lower) + '_' + str(upper) + '.csv')